In [52]:
from datasets import load_dataset
from googletrans import Translator, LANGUAGES
import pandas as pd
from pandarallel import pandarallel
import fasttext
import fasttext.util

In [32]:
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [39]:
translator = Translator()

In [42]:
LANGUAGES

{'af': 'afrikaans',
 'sq': 'albanian',
 'am': 'amharic',
 'ar': 'arabic',
 'hy': 'armenian',
 'az': 'azerbaijani',
 'eu': 'basque',
 'be': 'belarusian',
 'bn': 'bengali',
 'bs': 'bosnian',
 'bg': 'bulgarian',
 'ca': 'catalan',
 'ceb': 'cebuano',
 'ny': 'chichewa',
 'zh-cn': 'chinese (simplified)',
 'zh-tw': 'chinese (traditional)',
 'co': 'corsican',
 'hr': 'croatian',
 'cs': 'czech',
 'da': 'danish',
 'nl': 'dutch',
 'en': 'english',
 'eo': 'esperanto',
 'et': 'estonian',
 'tl': 'filipino',
 'fi': 'finnish',
 'fr': 'french',
 'fy': 'frisian',
 'gl': 'galician',
 'ka': 'georgian',
 'de': 'german',
 'el': 'greek',
 'gu': 'gujarati',
 'ht': 'haitian creole',
 'ha': 'hausa',
 'haw': 'hawaiian',
 'iw': 'hebrew',
 'he': 'hebrew',
 'hi': 'hindi',
 'hmn': 'hmong',
 'hu': 'hungarian',
 'is': 'icelandic',
 'ig': 'igbo',
 'id': 'indonesian',
 'ga': 'irish',
 'it': 'italian',
 'ja': 'japanese',
 'jw': 'javanese',
 'kn': 'kannada',
 'kk': 'kazakh',
 'km': 'khmer',
 'ko': 'korean',
 'ku': 'kurdish 

In [2]:
dataset = load_dataset('cnn_dailymail', split='train', name='3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /Users/intrro/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


In [10]:
data

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a
...,...,...,...
287108,"The nine-year-old daughter of a black, unarmed...","Rumain Brisbon, 34, was killed after Phoenix p...",279a12d3ee37b8109cc192a9e88115a5a631fb06
287109,Legalising assisted suicide is a slippery slop...,"Theo Boer, a European assisted suicide watchdo...",b5bc9d404a9a5d890c9fc26550b67e6d8d83241f
287110,A group calling itself 'The Women of the 99 Pe...,Ohio congressman criticised for 'condoning the...,500862586f925e406f8b662934e1a71bbee32463
287111,Most men enjoy a good pint of lager or real al...,The Black Country Ale Tairsters have been to 1...,32a1f9e5c37a938c0c0bca1a1559247b9c4334b2


In [26]:
train_data = pd.DataFrame(dataset)
sample_train_data = train_data.head()

In [27]:
sample_train_data

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


In [61]:
ft = fasttext.load_model('cc.ka.300.bin')

In [50]:
sample_train_data['geo_highlights'] = sample_train_data['highlights'].apply(lambda row: translator.translate(row, dest='ka').text)
sample_train_data['geo_article'] = sample_train_data['article'].apply(lambda row: translator.translate(row, dest='ka').text)

/var/folders/gk/9w8xz9lx5zj0176jtx7d4s0h0000gn/T/ipykernel_93751/401560289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_train_data['geo_highlights'] = sample_train_data['highlights'].apply(lambda row: translator.translate(row, dest='ka').text)
/var/folders/gk/9w8xz9lx5zj0176jtx7d4s0h0000gn/T/ipykernel_93751/401560289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_train_data['geo_article'] = sample_train_data['article'].apply(lambda row: translator.translate(row, dest='ka').text)


In [60]:
sample_train_data['geo_highlights_vector'] = sample_train_data['geo_highlights'].apply(lambda row:row.split())
sample_train_data['geo_article_vector'] = sample_train_data['geo_article'].apply(lambda row:row.split())

/var/folders/gk/9w8xz9lx5zj0176jtx7d4s0h0000gn/T/ipykernel_93751/2925248234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_train_data['geo_highlights_vector'] = sample_train_data['geo_highlights'].apply(lambda row:row.split())
/var/folders/gk/9w8xz9lx5zj0176jtx7d4s0h0000gn/T/ipykernel_93751/2925248234.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_train_data['geo_article_vector'] = sample_train_data['geo_article'].apply(lambda row:row.split())


In [63]:
sample_train_data['geo_highlights_vector'] = sample_train_data['geo_highlights'].apply(lambda row:[ft.get_word_vector(ob) for ob in row])
sample_train_data['geo_article_vector'] = sample_train_data['geo_article'].apply(lambda row:[ft.get_word_vector(ob) for ob in row])

/var/folders/gk/9w8xz9lx5zj0176jtx7d4s0h0000gn/T/ipykernel_93751/179033157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_train_data['geo_highlights_vector'] = sample_train_data['geo_highlights'].apply(lambda row:[ft.get_word_vector(ob) for ob in row])
/var/folders/gk/9w8xz9lx5zj0176jtx7d4s0h0000gn/T/ipykernel_93751/179033157.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_train_data['geo_article_vector'] = sample_train_data['geo_article'].apply(lambda row:[ft.get_word_vector(ob) for 

In [64]:
sample_train_data

,article,highlights,id,geo_highlights,geo_article,geo_highlights_vector,geo_article_vector
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4,ჰარი პოტერის ვარსკვლავი დენიელ რედკლიფი ორშაბა...,"ლონდონი, ინგლისი (როიტერი) -- ჰარი პოტერის ვარ...","[[-0.059350174, -0.09026988, -0.1698816, 0.051...","[[-0.044468567, -0.1750283, -0.11576537, 0.073..."
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9,"მაიამიში ფსიქიურად დაავადებული პატიმრები ""მივი...","რედაქტორის შენიშვნა: ჩვენს სერიებში, კულისებში...","[[-0.00038819277, -0.119878404, -0.1472499, 0....","[[0.018770188, -0.09589261, -0.080980316, 0.02..."
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37,"ახალი: „მეგონა მოვკვდებოდი“, ამბობს მძღოლი.\nკ...","მინეაპოლისი, მინესოტა (CNN) - მძღოლებმა, რომლე...","[[0.097816035, -0.11527624, -0.07165637, -0.03...","[[-0.00038819277, -0.119878404, -0.1472499, 0...."
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88,პროცედურის დროს აღმოჩენილი ხუთი პატარა პოლიპი;...,ვაშინგტონი (CNN) - ექიმებმა შაბათს პრეზიდენტ ბ...,"[[-0.14166206, -0.12782975, 0.00626321, -0.041...","[[-0.021756032, -0.13187249, -0.13848904, -0.1..."
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a,"ახალი: NFL-ის უფროსი, Atlanta Falcons-ის მფლობ...",(CNN) - ეროვნულმა საფეხბურთო ლიგამ განუსაზღვრე...,"[[0.097816035, -0.11527624, -0.07165637, -0.03...","[[-0.043408908, -0.15390097, -0.07080208, -0.0..."
